# COVID-19 Predictions with ARIMA model

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import random
import math
import time
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import operator 
from pmdarima.arima import auto_arima
import pmdarima as pm
from pmdarima.model_selection import train_test_split
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
from datetime import date
import operator
from matplotlib import pyplot

In [2]:
confirmed_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
coloco = ['Lat','Long']
confirmed_cases_global.drop(coloco, inplace=True, axis=1)
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Recovered']
confirmed_cases_global=confirmed_cases_global[confirmed_cases_global['Province/State']!='Diamond Princess']
confirmed_cases_global.head()


,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894
1,NaN,Albania,0,0,0,0,0,0,0,0,...,712,726,736,750,766,773,782,789,795,803
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,738,738,743,743,743,745,745,747,748,750
4,NaN,Angola,0,0,0,0,0,0,0,0,...,25,26,27,27,27,27,30,35,35,35


In [3]:
#create a row for Canada
confirmed_cases_Canada=confirmed_cases_global[confirmed_cases_global['Country/Region']=='Canada']
confirmed_cases_Canada=confirmed_cases_Canada.groupby(['Country/Region']).sum().reset_index()
confirmed_cases_Canada.insert(0, 'Province/State', float('NaN'))
confirmed_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Canada,0,0,0,0,1,1,2,2,...,46371,48033,49616,51150,52865,54457,56342,57925,60503,61956


In [4]:
confirmed_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region', 'Long_':'Long' }, inplace=True)
coloc = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key','Lat','Long']
confirmed_cases_US.drop(coloc, inplace=True, axis=1)
confirmed_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,141,141,141,141,141,145,145,145,145,145
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,14,14,14,14,14,14,14,14,14,14
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,1307,1371,1389,1400,1433,1539,1575,1757,1808,1843
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,55,57,57,57,57,66,66,66,66,66


In [5]:
confirmed_cases_US=confirmed_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()

In [6]:
deaths_cases_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
colo = ['Lat','Long']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Recovered']
deaths_cases_global=deaths_cases_global[deaths_cases_global['Province/State']!='Diamond Princess']
deaths_cases_global.drop(colo, inplace=True, axis=1)
deaths_cases_global.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,47,50,57,58,60,64,68,72,85,90
1,NaN,Albania,0,0,0,0,0,0,0,0,...,27,28,28,30,30,31,31,31,31,31
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,419,425,432,437,444,450,453,459,463,465
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,40,40,40,41,42,42,43,44,45,45
4,NaN,Angola,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [7]:
deaths_cases_Canada=deaths_cases_global[deaths_cases_global['Country/Region']=='Canada']
deaths_cases_Canada=deaths_cases_Canada.groupby(['Country/Region']).sum().reset_index()
deaths_cases_Canada.insert(0, 'Province/State', float('NaN'))
deaths_cases_Canada

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Canada,0,0,0,0,0,0,0,0,...,2570,2686,2840,2982,3154,3309,3536,3683,3794,4002


In [8]:
deaths_cases_US = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
deaths_cases_US.rename(columns={'Province_State':'Province/State', 'Country_Region':'Country/Region','Long_':'Long' }, inplace=True)
col = ['UID', 'iso2','iso3' ,'code3', 'FIPS','Admin2','Combined_Key', 'Population','Long','Lat']
deaths_cases_US.drop(col, inplace=True, axis=1)
deaths_cases_US.head()

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,American Samoa,US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,0,0,0,0,0,0,0,0,...,5,5,5,5,5,5,5,5,5,5
2,Northern Mariana Islands,US,0,0,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2
3,Puerto Rico,US,0,0,0,0,0,0,0,0,...,83,84,84,86,86,92,94,95,97,97
4,Virgin Islands,US,0,0,0,0,0,0,0,0,...,3,4,4,4,4,4,4,4,4,4


In [9]:
deaths_cases_US = deaths_cases_US.groupby(['Province/State','Country/Region']).sum().reset_index()


In [10]:
confirmed_cases= pd.concat([confirmed_cases_global, confirmed_cases_US,confirmed_cases_Canada], axis=0, sort=False)
confirmed_cases.to_csv('confirmed_cases_total.csv', sep=',',index = False)

In [11]:
deaths_cases = pd.concat([deaths_cases_global, deaths_cases_US,deaths_cases_Canada], axis=0, sort=False)
deaths_cases.to_csv('deaths_cases_total.csv', sep=',',index = False)

In [12]:
recoveries_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
coloco = ['Lat','Long']
recoveries_cases.drop(coloco, inplace=True, axis=1)
recoveries_cases.head()
recoveries_cases.to_csv('recoveries_cases_total.csv', sep=',',index = False)

In [13]:
colnames = confirmed_cases.columns.values

In [14]:
colonames = recoveries_cases.columns.values

In [15]:
confirmed = confirmed_cases.loc[:, colnames[4]:colnames[-1]]
deaths = deaths_cases.loc[:, colnames[4]:colnames[-1]]
recoveries = recoveries_cases.loc[:, colonames[4]:colonames[-1]]

In [16]:
time_date = confirmed.columns.values

In [17]:
all_days = np.array([i for i in range(len(time_date))]).reshape(-1, 1)

# Predictions for days ahead

In [18]:
pred_days_ahead = 2
#pred_days_ahead = 7
#pred_days_ahead = 30
predictions_ahead = np.array([i for i in range(len(time_date)+pred_days_ahead)]).reshape(-1, 1)

predictions_ahead_plus_day = np.array([i for i in range(len(time_date)+pred_days_ahead+1)]).reshape(-1, 1)

In [19]:
start = datetime.datetime.strptime(colnames[4], "%m/%d/%y")
end = datetime.datetime.strptime(colnames[-1], "%m/%d/%y")
predictions_ahead_dates = [start + datetime.timedelta(days=d) for d in range(len(predictions_ahead_plus_day))]

In [32]:
confirmed_cases = confirmed_cases[~confirmed_cases['Country/Region'].isin(['Tajikistan'])]

#confirmed_cases = confirmed_cases[confirmed_cases['Country/Region'].isin(['Tajikistan'])]
confirmed_cases

,Province/State,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,1463,1531,1703,1828,1939,2171,2335,2469,2704,2894
1,NaN,Albania,0,0,0,0,0,0,0,0,...,712,726,736,750,766,773,782,789,795,803
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,3256,3382,3517,3649,3848,4006,4154,4295,4474,4648
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,738,738,743,743,743,745,745,747,748,750
4,NaN,Angola,0,0,0,0,0,0,0,0,...,25,26,27,27,27,27,30,35,35,35
5,NaN,Antigua and Barbuda,0,0,0,0,0,0,0,0,...,24,24,24,24,24,24,25,25,25,25
6,NaN,Argentina,0,0,0,0,0,0,0,0,...,3780,3892,4003,4127,4285,4428,4532,4681,4783,4887
7,NaN,Armenia,0,0,0,0,0,0,0,0,...,1677,1746,1808,1867,1932,2066,2148,2273,2386,2507
8,Australian Capital Territory,Australia,0,0,0,0,0,0,0,0,...,106,106,106,106,106,106,106,106,106,107
9,New South Wales,Australia,0,0,0,0,3,4,4,4,...,2994,3002,3004,3016,3016,3025,3030,3035,3033,3035


In [21]:
countries=confirmed_cases['Country/Region'].tolist()
province=confirmed_cases['Province/State'].tolist()

In [22]:
prediction_models = dict()
prediction_models_confirmed = {'SVR': 0}
prediction_models_deaths = {'SVR': 0}
prediction_models_recoveries = {'SVR': 0}

In [23]:
# Prediction Models

from sklearn import model_selection
from sklearn.svm import SVR
def SVM_regression(train_x, train_y, test_x, test_y, X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
   # c = [0.01, 0.1, 1]
   # gamma = [0.01, 0.1, 1]
   #epsilon = [0.01, 0.1, 1]
   #shrinking = [True, False]
    #degree = [3, 4, 5, 6]
  #grid_parameters = {'C': c, 'gamma' : gamma ,'epsilon': epsilon, 'shrinking' : shrinking, 'degree':degree} 
   #regressor = RandomizedSearchCV(SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1, C=0.1), grid_parameters, scoring='neg_mean_squared_error', 
    #                               cv=5)
    
    if (subpopulation == 'confirmed'):
         degree = 3
    elif (subpopulation == 'deaths'):
        degree = 4
    else :
        degree = 5


    regressor = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=degree, C=0.1)
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['SVR'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['SVR'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['SVR'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions) 
    

In [24]:
from sklearn.ensemble import RandomForestRegressor
def RF_regression(train_x, train_y, test_x, test_y,X_test,subpopulation,X_train, Y_train):
    
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    grid_parameters = {'n_estimators': n_estimators}
#    regressor = RandomizedSearchCV(RandomForestRegressor(), grid_parameters, scoring='neg_mean_squared_error', 
#                                   cv=3)
    
    regressor = RandomForestRegressor(n_estimators = 200, random_state = 42) 
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Random_Forest'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)


In [25]:
# Gradient Boosted Regression Trees (GBRT) 
from sklearn.ensemble import GradientBoostingRegressor

def Grad_Boost_regression(train_x, train_y, test_x, test_y,X_test,subpopulation, X_train, Y_train):
   
    # Tune parameters to find optimal ones
    n_estimators = [100, 150, 300]
    loss = ['ls','lad','huber']
    grid_parameters = {'n_estimators': n_estimators}
  #  regressor = RandomizedSearchCV(GradientBoostingRegressor(learning_rate=0.1), grid_parameters, scoring='neg_mean_squared_error', 
 #                                  cv=3)
    
    regressor =  GradientBoostingRegressor(n_estimators=500, learning_rate=0.1,
                                            max_depth=1, random_state=0, loss='huber')
    regressor.fit(train_x, train_y)
    y_pred = regressor.predict(test_x)
    
    
    if (subpopulation == 'confirmed'):
         prediction_models_confirmed['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    elif (subpopulation == 'deaths'):
        prediction_models_deaths['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    else :
        prediction_models_recoveries['Grad_Tree_Boost'] = mean_absolute_error(y_pred, test_y)
    
    regressor.fit(X_train, Y_train)
    predictions = regressor.predict(X_test)
    
    return(predictions)



In [26]:
#check if a timeseries is stationary
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    rolmean = timeseries.rolling(12).mean()     
    rolstd = timeseries.rolling(12).std()
    
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    return dfoutput
    

In [27]:
def pred_with_arima(timeseries,n_days_ahead):
    
    #find the optimal d 
    diff=timeseries-timeseries.shift(1)
    for d in range(1,10):
        if (test_stationarity(diff.dropna())['p-value']<0.1):
            break
        else: 
            d=d+1 
            diff=diff-diff.shift(1)
            
    #use the arima model to make predictions, use original timeseries with optimal d:
    model = pm.auto_arima(timeseries, start_p=0, start_q=0,
                             max_p=4, max_q=4,
                              seasonal=False,
                             d=d, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=False)
    #print(model)
    forecasts, conf_int =model.predict(n_days_ahead,return_conf_int=True ,alpha=0.05)
    
    return forecasts , conf_int  # returns a tuple, array of forecasts and all conf int

In [28]:
def Arima_Analysis(region,pred_days_ahead):
   #create a dataframe for timeseries analysis
    date_rng = pd.date_range(start=colnames[4], end=colnames[-1], freq='D') #creates the days
    df = pd.DataFrame(date_rng, columns=['date'])
    df['cases']=np.transpose(region) #this is a dataframe with dates and cases.
    df = df.set_index('date')
    df=df[df['cases'] != 0] #remove zeroes, they do not help for analysis. 
    if (len(df)>1):
        if(df['cases'][-1]-df['cases'][0]>5):
            pred_ts,conf_int=pred_with_arima(df['cases'],pred_days_ahead) #this is an array converted to a list
        else:
            pred_ts=[df['cases'][-1]+1]*pred_days_ahead
            conf_int=[np.array([df['cases'][-1],df['cases'][-1]+2])]*pred_days_ahead
    else:
        pred_ts=np.zeros(pred_days_ahead)
        conf_int=[np.zeros(pred_days_ahead)] * pred_days_ahead
    return pred_ts, conf_int

In [29]:
import operator
def subpopulation_Analysis(cases,country,province, all_days,predictions_ahead,subpopulation, pred_days_ahead):
    
    pred  = list()
    if pd.isnull(province):
        region = cases.loc[(cases['Country/Region'] == country) & (pd.isnull(cases['Province/State']))]
    else :
        region = cases.loc[(cases['Country/Region'] == country) & (cases['Province/State'] == province)]
    print(region)
    
    reg =  np.array(region.loc[:, colnames[4]:colnames[-1]])
    region =  np.array(region.loc[:, colnames[4]:colnames[-1]]).reshape(-1, 1)
        
    #train_x, test_x, train_y, test_y = model_selection.train_test_split(all_days,region,test_size = 0.04,random_state = 42,shuffle=False)
    
    #pred_svm =  SVM_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_svm)
    
    #TODO: a for loop
    daily_increase_threshold1=np.absolute((reg[-1][-1]-reg[-1][-2])/reg[-1][-2])*100
    daily_increase_threshold2=np.absolute((reg[-1][-2]-reg[-1][-3])/reg[-1][-3])*100
    daily_increase_threshold3=np.absolute((reg[-1][-3]-reg[-1][-4])/reg[-1][-4])*100
    daily_increase_threshold4=np.absolute((reg[-1][-4]-reg[-1][-5])/reg[-1][-5])*100
    daily_increase_bound=1000
    daily_increase_threshold = max(daily_increase_threshold1,daily_increase_threshold2,daily_increase_threshold3,daily_increase_threshold4)
    
    if (daily_increase_threshold>daily_increase_bound):
        best_pred_ahead = 0  
        region = region
        best_pred = 0
        best_conf_int = np.zeros(2)
        reject_country = 1
        return(best_pred_ahead, region, best_pred, best_conf_int, reject_country)
     
    
    pred_arima, conf_int=Arima_Analysis(reg,pred_days_ahead)
    #pred.append(pred_svm)
    #pred_RF = RF_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
    #pred.append(pred_RF)

    
   # pred_gb =  Grad_Boost_regression(train_x, train_y.ravel(), test_x, test_y, predictions_ahead,subpopulation,all_days,region.ravel())
   # pred.append(pred_gb)


    if (subpopulation == 'confirmed'): 
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_confirmed.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    elif (subpopulation == 'deaths'):
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_deaths.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
    else :
        if (len(pred_arima)==0):
            min_idx, (min_key, min_val) = min(enumerate(prediction_models_recoveries.items()), key=lambda x: x[1][1])
            best_pred = pred[min_idx]
            conf_int=np.zeros(2)
        else:
            best_pred=pred_arima
            conf_int=conf_int
        

    best_pred_ahead = best_pred[-1]
    best_conf_int=conf_int[-1]
   
    best_conf_int[0]=max(reg[-1][-1], best_conf_int[0])
    if (best_conf_int[0]>best_conf_int[1]):
        best_conf_int[1]=2*reg[-1][-1]

    if(best_pred_ahead<reg[-1][-1]):
        best_pred_ahead= reg[-1][-1]+pred_days_ahead
    
    reject_country = 0
    
    return best_pred_ahead, region, best_pred, best_conf_int, reject_country

In [30]:
pred_file = str(2)+"day_prediction_" +  date.today().strftime('%Y-%m-%d') + "_arima.csv"
#pred_file = str(7)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + "_arima.csv"
#pred_file = str(30)+"day_prediction_" + date.today().strftime('%Y-%m-%d') + "_arima.csv"
pred_file

'2day_prediction_2020-05-05.csv'

In [31]:
columns = ['Province/State','Country','Target/Date','N','low95N','high95N',
          'R','low95R','high95R','D','low95D','high95D','T','low95T','high95T',
           'M','low95M','high95M','C','low95C','high95C']

            

k = 0
for co in range(len(countries)):
    
    ignore_recoveries = 0    
    
    total_predictions = pd.DataFrame(columns = columns)
    
    k = k + 1
       
    best_pred_confirmed, region_con, best_pred_con, conf_int_con, reject_country = subpopulation_Analysis(confirmed_cases,countries[co],province[co],
                                                 all_days,predictions_ahead,"confirmed",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_conf')
        continue
        
    best_pred_deaths, region_d, best_pred_d, conf_int_d, reject_country = subpopulation_Analysis(deaths_cases,countries[co],province[co],
                                              all_days,predictions_ahead,"deaths",pred_days_ahead)
    if (reject_country==1):
        print('rejected_d')
        continue
        
    if ((countries[co] == 'US' or countries[co] == 'Canada' ) & (pd.notnull(province[co]))):
        ignore_recoveries=1   
    else:
        best_pred_recoveries, region_r, best_pred_r, conf_int_r, reject_country = subpopulation_Analysis(recoveries_cases,countries[co],province[co],
                                                  all_days,predictions_ahead,"recoveries",pred_days_ahead)
    
    if (reject_country==1):
        print('rejected_r')
        continue
    
    # Mortality rate
    if (conf_int_con[0]==0):
        mort_pred=0
        mort_pred_low=0
        mort_pred_up=0
    else:
        mort_pred  = best_pred_deaths/best_pred_confirmed
        mort_pred_low= conf_int_d[0]/conf_int_con[1]
        mort_pred_up=conf_int_d[1]/conf_int_con[0]
    
    #fix province
    province_name=''
    if (pd.notnull(province[co])) :
        province_name=province[co]
    
    if (ignore_recoveries==0):
        R_value=int(best_pred_recoveries) 
        low_R=int(conf_int_r[0])
        high_R=int(conf_int_r[1])
    else:
        R_value=''
        low_R=''
        high_R=''
        
    #Critical Cases:
    #Assume that 5% of all cases are critical and need hospitalization:
    serious_percentage=0.05
    critical=serious_percentage*best_pred_confirmed
    low_critical=serious_percentage*conf_int_con[0]
    high_critical=serious_percentage*conf_int_con[1]
    
    total_predictions = total_predictions.append({'Province/State': province_name,
                                                'Country': countries[co],
                                                'Target/Date': predictions_ahead_dates[-1].strftime('%Y-%m-%d'), 
                                                'N': int(best_pred_confirmed) ,
                                                'low95N':int(conf_int_con[0]),
                                                'high95N': int(conf_int_con[1]),
                                                'R': R_value ,
                                                'low95R' : low_R,
                                                'high95R' : high_R,
                                                'D': int(best_pred_deaths) ,
                                                'low95D' : int(conf_int_d[0]),
                                                'high95D': int(conf_int_d[1]),
                                                'T': '',
                                                'low95T' : '' ,
                                                'high95T' : '',
                                                'M': mort_pred,
                                                'low95M': mort_pred_low,
                                                'high95M': mort_pred_up ,
                                                'C': int(critical) ,
                                                'low95C':  int(low_critical) ,
                                                'high95C': int(high_critical) },ignore_index = True)
    
    if (k==1):
        header = True
    else:
        header = False
    
    total_predictions.to_csv(pred_file, sep=',',index = False,  mode='a', header = header)
    

  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
0            NaN    Afghanistan        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  4/25/20  4/26/20  4/27/20  4/28/20  \
0        0        0        0  ...     1463     1531     1703     1828   

   4/29/20  4/30/20  5/1/20  5/2/20  5/3/20  5/4/20  
0     1939     2171    2335    2469    2704    2894  

[1 rows x 106 columns]
Fit ARIMA: (0, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=749.894, BIC=754.333, Time=0.016 seconds
Fit ARIMA: (0, 3, 1)x(0, 0, 0, 0) (constant=True); AIC=677.516, BIC=684.175, Time=0.163 seconds
Fit ARIMA: (0, 3, 2)x(0, 0, 0, 0) (constant=True); AIC=630.660, BIC=639.538, Time=0.167 seconds
Fit ARIMA: (0, 3, 3)x(0, 0, 0, 0) (constant=True); AIC=625.598, BIC=636.696, Time=0.378 seconds
Fit ARIMA: (0, 3, 4)x(0, 0, 0, 0) (constant=True); AIC=627.453, BIC=640.770, Time=0.510 seconds
Fit ARIMA: (1, 3, 0)x(0, 0, 0, 0) (constant=True); AIC=708.269, BIC=714.927, Tim

Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=377.698, BIC=387.825, Time=0.219 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=378.646, BIC=390.798, Time=0.379 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=380.665, BIC=394.842, Time=0.448 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=377.610, BIC=387.736, Time=0.125 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=379.606, BIC=391.758, Time=0.214 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=376.967, BIC=391.144, Time=0.552 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=379.597, BIC=391.749, Time=0.156 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=381.363, BIC=395.540, Time=0.536 seconds
Total fit time: 4.150 seconds
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
1            NaN        Albania        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  4/25/2

Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=311.454, BIC=321.211, Time=0.106 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=310.326, BIC=322.034, Time=0.141 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=308.734, BIC=314.588, Time=0.031 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=310.388, BIC=318.193, Time=0.063 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=311.537, BIC=321.293, Time=0.196 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=312.196, BIC=323.904, Time=0.193 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=312.224, BIC=325.883, Time=0.251 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=310.256, BIC=318.061, Time=0.055 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=310.773, BIC=320.530, Time=0.125 seconds
Fit ARIMA: (2, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=304.682, BIC=316.389, Time=0.443 seconds
Fit ARIMA: (2, 2, 3)x(0, 0, 0, 0) (const

Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=139.116, BIC=151.444, Time=0.195 seconds
Total fit time: 3.255 seconds
  Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
3            NaN        Andorra        0        0        0        0        0   

   1/27/20  1/28/20  1/29/20  ...  4/25/20  4/26/20  4/27/20  4/28/20  \
3        0        0        0  ...      344      344      385      398   

   4/29/20  4/30/20  5/1/20  5/2/20  5/3/20  5/4/20  
3      423      468     468     472     493     499  

[1 rows x 106 columns]
Fit ARIMA: (0, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=447.358, BIC=451.222, Time=0.016 seconds
Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=408.490, BIC=414.286, Time=0.137 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=408.548, BIC=416.275, Time=0.095 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=407.896, BIC=417.555, Time=0.282 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True);

Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=173.757, BIC=181.562, Time=0.121 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=176.206, BIC=185.962, Time=0.094 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=174.584, BIC=186.291, Time=0.176 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=174.554, BIC=188.212, Time=0.260 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=174.045, BIC=181.850, Time=0.055 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=176.025, BIC=185.781, Time=0.094 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=172.847, BIC=184.555, Time=0.427 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=173.948, BIC=187.607, Time=0.437 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=175.923, BIC=185.679, Time=0.062 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=175.129, BIC=186.836, Time=0.214 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (const

...

Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=439.973, BIC=454.275, Time=0.321 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=439.105, BIC=451.363, Time=0.156 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=440.512, BIC=454.813, Time=0.270 seconds
Total fit time: 3.237 seconds
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
51        Vermont             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  4/25/20  4/26/20  4/27/20  4/28/20  \
51        0        0        0  ...       46       46       47       47   

    4/29/20  4/30/20  5/1/20  5/2/20  5/3/20  5/4/20  
51       47       49      50      51      53      52  

[1 rows x 106 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=144.929, BIC=148.586, Time=0.016 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=146.810, BIC=152.296, Time=0.047 seconds
Fit ARIMA: (0, 1, 2)x(0, 0, 0, 0) (constant=

Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=362.841, BIC=368.577, Time=0.133 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=364.170, BIC=371.818, Time=0.226 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=364.860, BIC=374.420, Time=0.324 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=366.809, BIC=378.281, Time=0.400 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=385.874, BIC=391.610, Time=0.044 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=364.371, BIC=372.019, Time=0.236 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=365.857, BIC=375.417, Time=0.297 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=366.583, BIC=378.055, Time=0.312 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=367.953, BIC=381.337, Time=0.531 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=375.932, BIC=383.580, Time=0.084 seconds
Fit ARIMA: (2, 2, 1)x(0, 0, 0, 0) (const

Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=427.123, BIC=438.224, Time=0.293 seconds
Fit ARIMA: (3, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=428.062, BIC=441.013, Time=0.485 seconds
Fit ARIMA: (4, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=426.824, BIC=437.925, Time=0.252 seconds
Fit ARIMA: (4, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=428.747, BIC=441.698, Time=0.321 seconds
Total fit time: 4.389 seconds
   Province/State Country/Region  1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  \
55  West Virginia             US        0        0        0        0        0   

    1/27/20  1/28/20  1/29/20  ...  4/25/20  4/26/20  4/27/20  4/28/20  \
55        0        0        0  ...       32       34       34       38   

    4/29/20  4/30/20  5/1/20  5/2/20  5/3/20  5/4/20  
55       38       41      46      48      48      48  

[1 rows x 106 columns]
Fit ARIMA: (0, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=135.550, BIC=138.661, Time=0.016 seconds
Fit ARIMA: (0, 1, 1)x(0, 0, 0, 0) (constant=

Fit ARIMA: (1, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=457.238, BIC=463.148, Time=0.062 seconds
Fit ARIMA: (1, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=459.226, BIC=467.107, Time=0.131 seconds
Fit ARIMA: (1, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=461.153, BIC=471.004, Time=0.198 seconds
Fit ARIMA: (1, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=462.774, BIC=474.596, Time=0.348 seconds
Fit ARIMA: (1, 1, 4)x(0, 0, 0, 0) (constant=True); AIC=464.756, BIC=478.548, Time=0.482 seconds
Fit ARIMA: (2, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=459.178, BIC=467.059, Time=0.108 seconds
Fit ARIMA: (2, 1, 1)x(0, 0, 0, 0) (constant=True); AIC=461.140, BIC=470.992, Time=0.208 seconds
Fit ARIMA: (2, 1, 2)x(0, 0, 0, 0) (constant=True); AIC=462.594, BIC=474.416, Time=0.389 seconds
Fit ARIMA: (2, 1, 3)x(0, 0, 0, 0) (constant=True); AIC=464.762, BIC=478.554, Time=0.509 seconds
Fit ARIMA: (3, 1, 0)x(0, 0, 0, 0) (constant=True); AIC=461.025, BIC=470.877, Time=0.155 seconds
Fit ARIMA: (3, 1, 1)x(0, 0, 0, 0) (const

Fit ARIMA: (0, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1103.919, BIC=1111.102, Time=0.257 seconds
Fit ARIMA: (0, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1099.738, BIC=1109.316, Time=0.258 seconds
Fit ARIMA: (0, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1101.715, BIC=1113.688, Time=0.379 seconds
Fit ARIMA: (0, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1103.403, BIC=1117.769, Time=0.537 seconds
Fit ARIMA: (1, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1120.244, BIC=1127.427, Time=0.038 seconds
Fit ARIMA: (1, 2, 1)x(0, 0, 0, 0) (constant=True); AIC=1100.349, BIC=1109.926, Time=0.223 seconds
Fit ARIMA: (1, 2, 2)x(0, 0, 0, 0) (constant=True); AIC=1106.794, BIC=1118.767, Time=0.308 seconds
Fit ARIMA: (1, 2, 3)x(0, 0, 0, 0) (constant=True); AIC=1101.373, BIC=1115.740, Time=0.364 seconds
Fit ARIMA: (1, 2, 4)x(0, 0, 0, 0) (constant=True); AIC=1103.289, BIC=1120.050, Time=0.582 seconds
Fit ARIMA: (2, 2, 0)x(0, 0, 0, 0) (constant=True); AIC=1107.520, BIC=1117.098, Time=0.131 seconds
Fit ARIMA: (2, 2, 1)